# Imports

In [64]:
import pickle
from gensim.models import Word2Vec
import gensim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import pandas as pd
from sentence_transformers import util
import torch

from collections import defaultdict
import heapq

# Constants

In [134]:
RECIPES_WITH_RATINGS = "../samples/recipes_with_ratings.obj"

INDEX_TO_RECIPE_OBJ = "../samples/index_to_recipe_with_ratings.obj"
RECIPE_TO_INDEX_OBJ = "../samples/recipe_to_index_with_ratings.obj"

WORD2VEC_ING_VECTORS = "../word_embeddings_files/vectors/word2vec_ingredients_vectors.obj"
WORD2VEC_ING_CAT_VECTORS = "../word_embeddings_files/vectors/word2vec_ing_cat_vectors.obj"
WORD2VEC_ING_CAT_KEY_VECTORS = "../word_embeddings_files/vectors/word2vec_ing_cat_key_vectors.obj"

WORD2VEC_ING_CAT_TENSORS = "../word_embeddings_files/vectors/word2vec_ing_cat_tensors.pt"

# Load data

In [135]:
with open(RECIPES_WITH_RATINGS, "rb") as input_file:
    recipes_with_ratings = pickle.load(input_file)

with open(INDEX_TO_RECIPE_OBJ, "rb") as input_file:
    index_to_recipe = pickle.load(input_file)

with open(RECIPE_TO_INDEX_OBJ, "rb") as input_file:
    recipe_to_index = pickle.load(input_file)

In [136]:
with open(WORD2VEC_ING_CAT_VECTORS, "rb") as input_file:
    word2vec_ing_cat_vectors = pickle.load(input_file)

In [68]:
# word2vec_ing_cat_tensors = torch.load(WORD2VEC_ING_CAT_TENSORS, map_location=torch.device('cpu'))

# Create tensors

In [84]:
word2vec_ing_cat_vectors_sample = {}
for key, value in word2vec_ing_cat_vectors.items():
    if key in recipe_to_index.keys():
        word2vec_ing_cat_vectors_sample[key] = value

In [139]:
vectors = list(word2vec_ing_cat_vectors_sample.values())
vectors = [np.array(x).ravel() for x in vectors]

word2vec_ing_cat_tensors = torch.tensor(vectors, dtype=torch.float)

In [180]:
list_pos_to_recipe_id = {}
i = 0
for key in word2vec_ing_cat_vectors_sample.keys():
    list_pos_to_recipe_id[i] = key
    i += 1

## Save

In [70]:
# torch.save(word2vec_ing_cat_tensors, WORD2VEC_ING_CAT_TENSORS)

# Cosine similarities

In [226]:
def create_top_k_similar_vectors(tensors, list_pos_to_recipe_id, top_k=25):
    top_k=100
    top_scores = defaultdict()
    for list_pos, key in test.items(): #list_pos_to_recipe_id.items():
        if(list_pos % 1000 == 0):
            print("Iteration: ", list_pos)
        cos_scores = util.pytorch_cos_sim(word2vec_ing_cat_tensors[list_pos], 
                                          word2vec_ing_cat_tensors)
        top_scores[key] = torch.topk(cos_scores, k=top_k)
        
    ingredients_top100 = defaultdict(list)
    for list_pos, key in test.items():
        if(list_pos % 1000 == 0):
            print("Iteration: ", list_pos)
        ingredients_top100[key] = [(list_pos_to_recipe_id[k[0].item()],k[1].item()) 
                                     for k 
                                     in list(tuple(zip(top_scores[key][1][0], top_scores[key][0][0])))]
        
    return ingredients_top100

In [224]:
top_k=100
top_scores = defaultdict()
for list_pos, key in test.items(): #list_pos_to_recipe_id.items():
    if(list_pos % 1000 == 0):
        print("Iteration: ", list_pos)
    cos_scores = util.pytorch_cos_sim(word2vec_ing_cat_tensors[list_pos], 
                                      word2vec_ing_cat_tensors)
    top_scores[key] = torch.topk(cos_scores, k=top_k)

Iteration:  0


In [225]:
ingredients_top100 = defaultdict(list)
for list_pos, key in test.items():
    if(list_pos % 1000 == 0):
        print("Iteration: ", list_pos)
    ingredients_top100[key] = [(list_pos_to_recipe_id[k[0].item()],k[1].item()) 
                                 for k 
                                 in list(tuple(zip(top_scores[key][1][0], top_scores[key][0][0])))]

Iteration:  0


In [227]:
create_top_k_similar_vectors(word2vec_ing_cat_tensors, test, 25)

Iteration:  0
Iteration:  0


KeyError: 214534

In [222]:
list_pos_to_recipe_id[214534]

353794